In [ ]:
import sqlite3

# Database file path
DATABASE_PATH = 'ColivingMaster.db'  # e.g., 'simulated_data.db'

# Connect to the database
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

# Create Tenants table
cursor.execute("""
CREATE TABLE Tenants (
    TenantID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name VARCHAR(100) NOT NULL,
    Email VARCHAR(100) UNIQUE NOT NULL,
    Phone VARCHAR(15),
    MoveInDate DATE NOT NULL,
    MoveOutDate DATE
)
""")

# Create Employees table
cursor.execute("""
CREATE TABLE Employees (
    EmployeeID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name VARCHAR(100) NOT NULL,
    Role VARCHAR(50) NOT NULL,
    Contact VARCHAR(15)
)
""")

# Create Rooms table
cursor.execute("""
CREATE TABLE Rooms (
    RoomID INTEGER PRIMARY KEY AUTOINCREMENT,
    RoomType VARCHAR(50) NOT NULL,
    Availability BOOLEAN NOT NULL,
    Price DECIMAL(10, 2) NOT NULL,
    EmployeeID INTEGER,
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
)
""")

# Create Reservations table
cursor.execute("""
CREATE TABLE Reservations (
    ReservationID INTEGER PRIMARY KEY AUTOINCREMENT,
    TenantID INTEGER NOT NULL,
    RoomID INTEGER NOT NULL,
    StartDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    FOREIGN KEY (TenantID) REFERENCES Tenants(TenantID),
    FOREIGN KEY (RoomID) REFERENCES Rooms(RoomID)
)
""")

# Create Events table
cursor.execute("""
CREATE TABLE Events (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    EventName VARCHAR(100) NOT NULL,
    Date DATE NOT NULL,
    Location VARCHAR(100),
    RoomID INTEGER,
    EmployeeID INTEGER,
    StartTime TIME NOT NULL,
    EndTime TIME NOT NULL,
    FOREIGN KEY (RoomID) REFERENCES Rooms(RoomID),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
)
""")

# Create Participation table
cursor.execute("""
CREATE TABLE Participation (
    ParticipationID INTEGER PRIMARY KEY AUTOINCREMENT,
    TenantID INTEGER NOT NULL,
    EventID INTEGER NOT NULL,
    Status VARCHAR(20) NOT NULL DEFAULT 'Confirmed',
    FOREIGN KEY (TenantID) REFERENCES Tenants(TenantID),
    FOREIGN KEY (EventID) REFERENCES Events(EventID)
)
""")

conn.commit()
cursor.close()
conn.close()

print("Tables created!")

OperationalError: table Tenants already exists